In [0]:
# Widgets
dbutils.widgets.text("sub", "", "Subestação")
dbutils.widgets.text("feeder", "", "Alimentador")
dbutils.widgets.text("uni_tr_mt", "", "Rede BT")
dbutils.widgets.text("random_state", "", "Semente Aleatória")
sub = dbutils.widgets.get("sub")
feeder = dbutils.widgets.get("feeder")
uni_tr_mt = dbutils.widgets.get("uni_tr_mt")
random_state = dbutils.widgets.get("random_state")
if random_state == "":
    random_state = None
else:
    random_state = int(random_state)

In [0]:
import os
import sys
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)

In [0]:
from collections import defaultdict
import json
import pandas as pd

In [0]:
# MDMS engine
import numpy as np
from Utils.MDMS import MDMS, config
mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)

In [0]:
data = {x.Name: x.Bus1.split(".")[0].split("_")[-1] for x in mdms.altdss.Load if x.Class%10 == 1}
seen = {}
result = {}
for key, value in data.items():
    if value not in seen:
        seen[value] = key
        result[key] = value
customers = result  
if len(customers) > 50:
    keys = np.random.choice(list(customers.keys()), min(100, len(customers)), replace=False)
    customers = {k: data[k] for k in keys if k in data}

In [0]:
cases = []
for customer in customers:
    kw = np.round(np.random.uniform(low=1.50, high=3.00, size=1), 2)
    pf = np.round(np.random.uniform(low=0.85, high=1.00, size=1), 2)
    cases.append({
        "name": [customer],
        "kw_by_phase": kw,
        "pf": pf,
        "shape_type": ["fixed"],
        "working_period": [(15, 21)],
        "theft_on_training": [False],
    })


In [0]:
# Medições
meas = mdms.acquire() 

# MLP
from Models.MultilayerPerceptron import MLP, Training
engine = Training(
    model=MLP(
        input_size=mdms.inputs.shape[-1],
        hidden_layers=list(np.asarray(config.mlp.hidden_layers)*mdms.inputs.shape[-1]),
        output_size=mdms.outputs.shape[-1],
        activation_function=config.mlp.activation_function
    ), 
    inputs = mdms.training.get("inputs"), 
    outputs = mdms.training.get("outputs"), 
    random_state=random_state
)
mlp = engine.train(
    log_params={
        "sub": sub,
        "feeder": feeder,
        "uni_tr_mt": uni_tr_mt,
        "random_state": random_state
    },
    use_mlflow = False
)

# LR
from sklearn.linear_model import LinearRegression
from Models.LinearRegression import Training
model = LinearRegression()
inputs, outputs = mdms.training.get("inputs"), mdms.training.get("outputs") 
engine = Training(
    model=model, 
    inputs=inputs, 
    outputs=outputs,
    test_size=0.3,
    random_state=random_state
)
linear_reg = engine.train()
mse, r2 = engine.evaluate_model()
print(f'MSE: {mse:.6f}, R²: {r2:.6f}')


In [0]:
def sql_literal(v):
    # Numéricos
    if isinstance(v, (int, float)):
        return str(v)

    # Booleanos
    if isinstance(v, bool):
        return "TRUE" if v else "FALSE"

    # Strings
    v = str(v)
    v = v.replace("'", "''")      
    return f"'{v}'"                     

In [0]:
from Models.MultilayerPerceptron import NTLDetection
from Models.LinearRegression import Detector
output = defaultdict(list)
for i, case_ in enumerate(cases):
    
    # Restart mdms engine
    mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)

    # NTL
    meas = mdms.acquire(customers=case_) 

    # Detecção MLP
    MLPdeltaW = NTLDetection(
        model = mlp,
        inputs = mdms.inference.get("inputs"),
        outputs = mdms.inference.get("outputs"),
    ).execute(
        rn_threshold=6,
        r2_threshold=0.90,
    )

    # LR
    inputs, outputs = mdms.inference.get("inputs"), mdms.inference.get("outputs") 
    LRdeltaW = Detector(model=linear_reg, inputs=inputs, outputs=outputs).execute()

    # Params
    output = {
        "sub": sub,
        "feeder": feeder,
        "uni_tr_mt": uni_tr_mt,
        "random_state": random_state,
        "irregular_bus": mdms.irregs.bus_id[0],
        "irregular_load": mdms.irregs.name[0],
        "phases": mdms.irregs.num_phases[0],
        "bypassed_kw": mdms.irregs.kw_by_phase[0],
        "bypassed_pf": mdms.irregs.pf[0],
        "kwh_measured": mdms.irregs.kwh_measured[0],
        "kwh_theft": mdms.irregs.kwh_theft[0],
        "mlp_suspects": json.dumps(dict(zip(MLPdeltaW.bus_id.tolist(), MLPdeltaW.deltaW.tolist()))),
        "lr_suspects": json.dumps(dict(zip(LRdeltaW.bus_id.tolist(), LRdeltaW.deltaW.tolist())))
    }
    data = pd.DataFrame(output, index=[0])
    data = spark.createDataFrame(data)
    data.write.format("delta").mode("append").option("overwriteSchema", "true").saveAsTable("le41.bronze.phd_monte_carlo_single_ntl")

